In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import statsmodels
from statsmodels.tsa.stattools import coint
import matplotlib.pyplot as plt
import dask
import datetime as dt
import dask.dataframe as dd

/Users/ppm/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)
/Users/ppm/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, n_workers=4, memory_limit='1.1GB')
client

Client Scheduler: tcp://127.0.0.1:57345 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 4.40 GB


In [3]:
datalist = pd.read_csv('companylist.csv')

In [4]:
datalisttech = datalist.loc[datalist['Sector'] == 'Technology']
datalisttechbillion = datalisttech.loc[datalisttech['MarketCap'].str.contains("B", na=False)]

In [5]:
tickerlist = datalisttechbillion['Symbol'].to_list()

In [6]:
#pandas_data = yf.download(tickers = tickerlist, period = '60d', interval = '2m', group_by='ticker', prepost=False)

In [7]:
#for stock in tickerlist:
#    pandas_data[stock].to_csv('stockdata/'+stock+'.csv')

In [9]:
#tryit = pd.read_csv('stockdata/AAPL.csv')

In [10]:
def equalizer(list1, list2):
    list1removal = []
    for date in list1:
        if date in list2:
            None
        else:
            list1removal.append(date)
    list2removal = []
    for date2 in list2:
        if date2 in list1:
            None
        else:
            list2removal.append(date2)
    return list1removal, list2removal

In [11]:
def single_coint_test(stock1, stock2):
        s1 = pd.read_csv('stockdata/'+stock1+'.csv', index_col = 'Datetime')['Open'].dropna()
        s2 = pd.read_csv('stockdata/'+stock2+'.csv', index_col = 'Datetime')['Open'].dropna()
        s1dates = s1.index.to_list()
        s2dates = s2.index.to_list()
        s1removal, s2removal = equalizer(s1dates, s2dates)
        pvaluetable = []
        for date in s1removal:
            s1.drop(date, inplace=True)
        for date2 in s2removal:
            s2.drop(date2, inplace=True)
        if len(s1.to_list()) <= 2 or len(s2.to_list()) <= 2:
            pvaluetable.append([stock1, stock2, 1.0, 0])
        elif len(s1.to_list()) != len(s2.to_list()):
            pvaluetable.append([stock1, stock2, 1.0, 0])
        else:
            score, pvalue, _ = coint(s1, s2, autolag='aic')
            if pvalue <= 0.01:
                pvaluetable.append([stock1, stock2, pvalue, 1])
            else:
                pvaluetable.append([stock1, stock2, pvalue, 0])
        del s1
        del s2
        del s1removal
        del s2removal
        return pvaluetable       

In [12]:
def listshift(ticker_list):
    newlist = [ticker_list[-1]] + ticker_list[:-1]
    return newlist

In [13]:
def dask_coint(tickers, rotations):
    dask_coint = dask.delayed(single_coint_test)
    dask_tasks_list = []
    newlist = tickerlist
    for count in np.arange(0, rotations):
        newlist = listshift(newlist)
        for stock_1, stock_2 in zip(tickerlist, newlist):
            task = dask_coint(stock_1, stock_2)
            dask_tasks_list.append(task)
    dask_persist_list = dask.persist(*dask_tasks_list)
    computations = dask.compute(dask_persist_list)
    return computations

In [14]:
def tasktime():
    starttime = dt.datetime.now()
    taskfile = dask_coint(tickerlist, 2)
    endtime = dt.datetime.now()
    time_elapsed = (endtime-starttime)
    return taskfile, time_elapsed

In [15]:
result, timer = tasktime()

In [21]:
len(result[0])

436

In [25]:
wow = pd.DataFrame(result[0])

In [26]:
wow.to_csv('results.csv')